In [ ]:
# Why noy use linear regression

# Logistic hypothesis

# Parameter estimation

### IMPORTAÇÕES NECESSÁRIAS

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer

%matplotlib widget

### DESCRIÇÃO DO PROBLEMA

Suponha que temos uma relação de pacientes com câncer de mama e precisamos desenvolver um modelo de Aprendizado de Máquina para prever qual a gravidade do câncer destes pacientes. Entre as respostas possíveis, temos que classificar o câncer como **maligno** ou **benigno**.

Dessa forma, o problema não se encaixa como um problema de regressão, mas sim de classificação. Ou seja, queremos prever uma variável qualitativa, como uma categoria (**maligno** ou **benigno**).

Para que os computadores possam "entender" as variáveis qualitativas, é preciso traduzi-las para a linguagem numérica, ex.: benigno = 0; e maligno = 1. Este processo de tradução é chamado de ***encoding*** ou **codificação**.

Veremos mais à frente que estes valores qualitativos podem ser interpretados como a probabilidade de uma determinada amostra pertencer a classe-alvo.

### CARREGAMENTO E VISUALIZAÇÃO DOS DADOS

In [ ]:
data = load_breast_cancer(as_frame=True)

print(f"{'-'*80}\nVariáveis independentes: {data['feature_names']} \n{'-'*80}\nVariáveis dependentes: {data['target_names']}")

In [ ]:
print(f'{"-"*100} \nX:')
display(data['data'].head(5))

print(f'{"-"*100} \ny:')
display(data['target'][-10:])

Ao analisar a base de dados, observa-se que o valor correspondente à classe **maligno** é 0 e aquele correspondente à classe **benigno** é 1. É comum em um problema de classificação binária (duas classes-alvo) que a classe principal a qual se deseja prever seja representada pelo número 1. Dessa forma, por questões didáticas, decidi trocar os valores entre as classes. Ou seja, agora o valor referente à classe **maligno** (classe principal) será 1 e o da classe **benigno** será 0.

**obs: essa mudança de ordem não altera em nada o processo de desenvolvimento e otimização do modelo de regressão logística.**

In [ ]:
X, y = data['data'].values, data['target'].values

y = np.where(y == 0, 1, 0)

In [ ]:
fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.scatter(X[:,2][y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='benigno')
ax0.scatter(X[:,2][y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='maligno')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.legend()

fig.tight_layout()
plt.show()

### POR QUE NÃO USAR REGRESSÃO LINEAR?

Uma vez transformadas as classes em valores binários 0/1, podemos treinar um regressor linear para estimar qual a categoria do câncer de um determinado paciente da seguinte forma:

In [ ]:
x_ = np.linspace(X[:,2].min(), X[:,2].max(), 10)
y_ = x_ * (1 / 40) - 2 # Linear Regressor

fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.scatter(X[:,2][y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(X[:,2][y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0.plot(x_, y_, linestyle='--', c='black', label='Regressor Linear')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.legend()

fig.tight_layout()
plt.show()

Analisando a figura anterior, podemos interpretar os valores preditos pelo regressor linear como uma estimativa de $P(y=1|X)$. Isto é, a probabilidade de uma determinada amostra $X$ pertencer a classe 1 (maligno). Observa-se que os valores estimados pelo regressor linear ultrapassam os limites de probabilidae (0 e 1) e podem variar de $-\infty$ a $+\infty$.

Portanto, é por essas e outras razões que é preferível a utilização de um regressor mais adequado capaz de lidar com valores qualitativos, como o modelo de **Regressão Logística**

### REGRESSÃO LOGÍSTICA

O modelo de Regressão Logística tem esse nome devido à função que serve de hipótese para o modelo: função logística, mais especificamente a função sigmóide.

A imagem da função sigmóide varia entre os valores 0 e 1. Sua equação é descrita da seguinte forma:

$$g(z) = \frac{1}{1 + e^{-z}}$$

Veremos mais à frente que, quando utilizada para modelar um problema de classificação do tipo $P(y=1|X)$, a função precisa ser ajustada e $z$ passa a ser uma abreviação de $w \cdot X + b$ e a versão completa da função é, então:

$$ g_{W, b}(X) = \frac{1}{1 + e^{-(W \cdot X + b)}} $$

onde $g_{W, b}(X) = P(y=1|X)$ é a variável dependente (saída), $X$ são as variáveis independentes (entrada), $W$ e $b$ são parâmetros a serem otimizados.

**OBS: para facilitar, a partir de agora, vamos abreviar $g_{W, b}(X)$ por $\hat{y}$, onde entende-se que $g(.)$ é a resposta calculada pelo modelo ($g(.)=\hat{y}$)**

A forma em "S" da função sigmoide pode ser visualizada na figura a seguir:

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
z = np.linspace(-10, 10, 100)
g = sigmoid(z)

fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.plot(z, g, c='black', label='Sigmoid')
ax0.set_xlabel('$z$')
ax0.set_ylabel('$g(z)$')
ax0.legend()

fig.tight_layout()
plt.show()

Agora vamos tentar implementar a função logística para receber como entrada as variáveis do nosso problema de classificação de câncer de mama e ver como os parâmetros influenciam a função.

In [ ]:
x_ = np.linspace(-50, X[:,2].max(), 1000)

w0, b0 = 1, 0
w1, b1 = 1, -5
w2, b2 = 0.05, 0
w3, b3 = 0.05, -5

z0 = x_ * w0 + b0
z1 = x_ * w1 + b1
z2 = x_ * w2 + b2
z3 = x_ * w3 + b3

g0 = sigmoid(z0)
g1 = sigmoid(z1)
g2 = sigmoid(z2)
g3 = sigmoid(z3)

fig = plt.figure(figsize=(7,5))

ax0 = fig.add_subplot(1,1,1)

ax0.scatter(X[:,2][y==0], y[y==0], c='green', ec='black', s=80, marker='o', label='Benigno')
ax0.scatter(X[:,2][y==1], y[y==1], c='red', ec='black', s=80, marker='X', label='Maligno')
ax0.plot(x_, g0, linestyle='--', c='black', label=f'w={w0} | b={b0}')
ax0.plot(x_, g1, linestyle='--', c='orange', label=f'w={w1} | b={b1}')
ax0.plot(x_, g2, linestyle='--', c='purple', label=f'w={w2} | b={b2}')
ax0.plot(x_, g3, linestyle='--', c='blue', label=f'w={w3} | b={b3}')
ax0.set_xlabel(data['feature_names'][2])
ax0.set_ylabel('Class')
ax0.legend()

fig.tight_layout()
plt.show()

### OTIMIZAÇÃO DOS PARÂMETROS

Quando desejamos modelar um regressor logístico para um problema de classificação, estimamos uma função logística cujos coeficientes $W$ e $b$ são desconhecidos. Portanto, é necessário estimar os coeficientes de maneira que a curva melhor se ajuste aos dados de treinamento.

Para a otimização dos coeficientes do modelo de Regressão Linear, é utilizado o método dos Mínimos Quadrados para minimizar a diferença entre os dados preditos $\hat{y}$ e reais $y$ da seguinte forma:

$$ MSE = \frac{1}{2m} \sum_{i=0}^{m-1} (\hat{y_{i}} - y_{i})^{2} $$

A utilização da MSE como função custo da Regressão Logística não é indicada e abaixo demonstraremos o porquê:

In [ ]:
def mse(y_pred, y):
    return np.mean((y_pred - y)**2, axis=0) / 2

In [ ]:
# utilizar utils

def standardize(x):
    return (x - x.mean(axis=0)) / x.std(axis=0)

In [ ]:
n_grid = 25

w_ = np.linspace(-25, 25, n_grid).reshape(1,-1)
b_ = np.linspace(-25, 25, n_grid).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

x_ = standardize(X[:, 2].reshape(-1,1))
z_ = np.dot(x_, W_.reshape(1,-1)) + B_.reshape(1,-1)
y_pred = sigmoid(z_)

mse_ = mse(y_pred, np.tile(y.reshape(-1,1), n_grid**2))
mse_ = mse_.reshape(n_grid, n_grid)

fig = plt.figure(figsize=(5,5))

ax0 = fig.add_subplot(111, projection='3d')
ax0.plot_surface(W_, B_, mse_, cmap='viridis')
ax0.set_xlabel('$W$')
ax0.set_ylabel('$b$')
ax0.set_zlabel('$MSE$')

fig.tight_layout()
plt.show()

A função MSE, quando utilizada para a Regressão Logística, assume uma forma não-convexa. Isto é, existem vários pontos da função custo diferentes do mínimo global que também possuem derivadas parciais com relação aos parâmetros iguais a 0. Dessa forma, a otimização dos parâmetros através do metódo Gradiente Descendente é prejudicada, uma vez que quando o gradiente é nulo não há atualização.

In [ ]:
z = np.linspace(-10, 10, 100)
g = sigmoid(z)

fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(1,1,1)

ax0.plot(z, g, c='black', label='Sigmoid')
ax0.set_xlabel('$z$')
ax0.set_ylabel('$g(z)$')
ax0.legend()

fig.tight_layout()
plt.show()

Para contornar esse problema, na Regressão Logística é utilizada a função logarítimica para estimar o erro de um determinado modelo, da seguinte forma:

$$ J(y, \hat{y}) = \begin{equation*}
        \begin{cases}
            \frac{1}{m} \sum_{i=0}^{m-1} -log(\hat{y}_{i}) \quad\quad\quad \text{   se } y_{i} = 1 \\ \\
            \frac{1}{m} \sum_{i=0}^{m-1} -log(1 - \hat{y}_{i}) \quad\space \text{   se } y_{i} = 0
        \end{cases}
    \end{equation*} $$

In [ ]:
def log_loss_(y, y_pred, c=0):
    if c==0:
        return -np.log(1 - y_pred[y==c]).mean(axis=0)
    elif c==1:
        return -np.log(y_pred[y==c]).mean(axis=0)

In [ ]:
w_ = np.ones(n_grid).reshape(1,-1) * 15

z_ = np.dot(x_, w_) + b_
y_pred = sigmoid(z_)

loss0 = log_loss_(y, y_pred, 0)
loss1 = log_loss_(y, y_pred, 1)

fig = plt.figure(figsize=(5,4))

ax0 = fig.add_subplot(111)
ax0.plot(b_.flatten(), loss0, label='$y=0$')
ax0.plot(b_.flatten(), loss1, label='$y=1$')
ax0.set_xlabel('$b$')
ax0.set_ylabel('$L(y, \hat{y})$')
ax0.legend()

fig.tight_layout()
plt.show()

Agora, podemos juntar as duas expressões em uma só, da seguinte forma:

$$ J(y, \hat{y}) = - \frac{1}{m} \sum_{i=0}^{m-1} (y) \cdot log(\hat{y_{i}}) + (1 - y) \cdot log(1 - \hat{y_{i}}) $$

In [ ]:
def log_loss(y, y_pred):
    return - np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred), axis=0)

In [ ]:
n_grid = 25

w_ = np.linspace(-10, 10, n_grid).reshape(1,-1)
b_ = np.linspace(-10, 10, n_grid).reshape(1,-1)
W_, B_ = np.meshgrid(w_, b_)

x_ = standardize(X[:, 2].reshape(-1,1))
z_ = np.dot(x_, W_.reshape(1,-1)) + B_.reshape(1,-1)
y_pred = sigmoid(z_)

loss_ = log_loss(np.tile(y.reshape(-1,1), n_grid**2), y_pred)
loss_ = loss_.reshape(n_grid, n_grid)

fig = plt.figure(figsize=(5,5))

ax0 = fig.add_subplot(111, projection='3d')
ax0.plot_surface(W_, B_, loss_, cmap='viridis')
ax0.set_xlabel('$W$')
ax0.set_ylabel('$b$')
ax0.set_zlabel('$J(y, \hat{y})$')

fig.tight_layout()
plt.show()

A função de perda logarítimica 